### Data Wrangling

- The following data wrangling processes uses pandas for analysis and manipulation of the dataset

#### The following are the required dependencies

In [5]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#### Storing the dataset as a local variable

- Similar to the data preview notebook, the pandas object structures the dataset into a format ready for preprocessing

In [6]:
data_frame = pd.read_csv("real_estate_sydney.csv")

#### Null Values

- Null values occur when a row of data does not include a particulat input feature, which can cause runtime errors and unexpected results/
- `isnull().sum` method shows if there are null values in any of the columns

In [7]:
data_frame.isnull().sum()

Target                      0
date_sold                   0
suburb                      0
num_bath                    0
num_bed                     0
num_parking                 0
property_size               0
type                        0
suburb_population           0
suburb_median_income        0
suburb_sqkm                 0
suburb_lat                  0
suburb_lng                  0
suburb_elevation            0
cash_rate                   0
property_inflation_index    0
km_from_cbd                 0
dtype: int64

- This is a high quality dataset, and there are no null values
- However for future reference, if there is null values, here are two common approaches

1. Remove a row with null values using `dropna()`
2. Replace null value with mean value for particular feature or target using `fillna()`

#### Removing Duplicates

- Duplicate data can reduce diversity and laead to overfitting and thus bias

- The `duplicated().sum()` method, returns the amount of duplicate rows

In [8]:
data_frame.duplicated().sum()

np.int64(0)

- Again this ia a high quality dataset meaning, there are no duplicates
- The `drop_duplicated()` method can be used to remove the duplicated rows

#### Replacing data

- The dataset has input features which are strings, meaning that `.apply()` functions can be used to whole columns to modify values
- Though first I want to view the amount of unique property types using the `unique()` function


In [9]:
data_frame['type'].unique()

array(['House', 'Vacant land', 'Townhouse', 'Apartment / Unit / Flat',
       'Semi-Detached', 'New House & Land', 'Duplex', 'Villa', 'New land',
       'Terrace', 'Studio', 'Block of Units', 'Development Site',
       'Acreage / Semi-Rural', 'New Apartments / Off the Plan', 'Rural'],
      dtype=object)

- This is far too many types of properties, before further wrangling, these property type features will be changed to either house or apartment, or removed altogether

In [10]:
# Extracts all of the type input features to be maniplated
types = data_frame['type'].to_list()

# Choosing options for the type of properties
house = ['House', 'Villa', 'Townhouse', 'Duplex', 'Terrace']
apartments = ['Apartment / Unit / Flat', 'Studio']

# Creating a new list to store the new types
new_types = []

# Looping through the types and checking if they are in the house or apartment list
# Property types do not include land as does not fit into the scope
for _ in types:
    if _ in house:
        new_types.append('House')
    elif _ in apartments:
        new_types.append('Apartment')
    else:
        new_types.append(None) # Rows that aren't houses or apartments are classified as None and are deleted later


data_frame['type'] = new_types

data_frame.dropna(subset=['type'], inplace=True)

- The dataset has 17 features, many of which will be removed to improve the simplicity of the dataset, and to allow room for engineered features

In [11]:
data_frame.drop(columns=['suburb','suburb_population', 'suburb_sqkm','cash_rate','property_inflation_index', 'suburb_elevation'], inplace=True)

#### Removing outliers

- Outliers can negatively affect accuracy in numerical columns

- The 25th and 75th quartiles can be used to estimate an appropraite range, and all that do not fit can be filtered out

- Mathematically speaking, outliers are values occuring 1.5x outside the interquartile range from Q1 or Q3



- However, miniscule sizes have been recorded, so the lower quartile will be adjusted from 1.5 to 1.3

In [12]:
#get the inter-quartile range on the property size column
print(data_frame['property_size'].describe())
Q1 = data_frame['property_size'].quantile(0.25)
Q3 = data_frame['property_size'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a property_size above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.3}')

count    10731.000000
mean       706.933464
std        975.025893
min          7.000000
25%        449.000000
50%        601.000000
75%        765.000000
max      59100.000000
Name: property_size, dtype: float64
Outliers are a property_size above 1239.0 or below 38.19999999999999


- The property sizes will be filtered within an acceptable range

In [13]:
# Filter salaries within the acceptable range
data_frame = data_frame[(data_frame['property_size'] >= Q1 - 1.3 * IQR) & (data_frame['property_size'] <= Q3 + 1.5 * IQR)]
print(data_frame['property_size'].describe())

count    10069.000000
mean       575.651008
std        255.499836
min         40.000000
25%        421.000000
50%        586.000000
75%        721.000000
max       1239.000000
Name: property_size, dtype: float64


- Looking for outliers in property sizes minimises the amount of outliers in all other features such as bedrooms and bathrooms, as it provides the most wholistic account of amount of bedrooms, bathrooms etc

In [14]:
#get the inter-quartile range on the distance from CBD column
print(data_frame['km_from_cbd'].describe())
Q1 = data_frame['km_from_cbd'].quantile(0.25)
Q3 = data_frame['km_from_cbd'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a km_from_cbd above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')

count    10069.000000
mean        26.792673
std         18.444099
min          0.310000
25%         12.680000
50%         21.790000
75%         40.120000
max         84.790000
Name: km_from_cbd, dtype: float64
Outliers are a km_from_cbd above 81.28 or below -28.479999999999997


- The minimum is extremely low, so here the Q1 outliers will be ignored because it is still valid data, unlike previous data

In [15]:
# Filter distance from cbd within the acceptable range
data_frame = data_frame[(data_frame['km_from_cbd'] <= Q3 + 1.5 * IQR)]
print(data_frame['km_from_cbd'].describe())

count    10018.000000
mean        26.506262
std         18.047546
min          0.310000
25%         12.680000
50%         21.620000
75%         39.760000
max         78.540000
Name: km_from_cbd, dtype: float64


#### Scaling features to a common range

- Scaling features optimises the data, allowing for the ease of machine learning algorithm to find an optimal solution as difference of scales does not influence them

In [16]:
scale_feature = 'property_size'

#the minimum value with space for outliers (With a small buffer)
MIN_PS = 40

#the maximum value with space for outliers (With a small buffer)
MAX_PS = 1270

#scale features
data_frame[scale_feature] = [(X - MIN_PS) / (MAX_PS - MIN_PS) for X in data_frame[scale_feature]]

data_frame.describe()

,Target,num_bath,num_bed,num_parking,property_size,suburb_median_income,suburb_lat,suburb_lng,km_from_cbd
count,1.001800e+04,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000
mean,1.633422e+06,2.039629,3.721701,1.950988,0.435324,40308.808545,-33.785384,151.101469,26.506262
std,1.100330e+06,1.018020,1.233622,1.216215,0.208062,11206.210816,0.198342,0.206479,18.047546
min,2.725000e+05,0.000000,0.000000,0.000000,0.000000,14248.000000,-34.106240,150.553840,0.310000
25%,1.000000e+06,1.000000,3.000000,1.000000,0.308943,32500.000000,-33.921480,150.956810,12.680000
50%,1.360000e+06,2.000000,4.000000,2.000000,0.444715,39260.000000,-33.813780,151.115060,21.620000
75%,2.000000e+06,2.000000,4.000000,2.000000,0.553659,45552.000000,-33.723380,151.227530,39.760000
max,1.700000e+07,46.000000,47.000000,50.000000,0.974797,97500.000000,-33.211420,151.563810,78.540000


In [17]:
scale_feature = 'km_from_cbd'

#the minimum value with space for outliers
MIN_KM = 0.3

#the maximum value with space for outliers
MAX_KM = 82

#scale features
data_frame[scale_feature] = [(X - MIN_KM) / (MAX_KM - MIN_KM) for X in data_frame[scale_feature]]

data_frame.describe()

,Target,num_bath,num_bed,num_parking,property_size,suburb_median_income,suburb_lat,suburb_lng,km_from_cbd
count,1.001800e+04,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000,10018.000000
mean,1.633422e+06,2.039629,3.721701,1.950988,0.435324,40308.808545,-33.785384,151.101469,0.320762
std,1.100330e+06,1.018020,1.233622,1.216215,0.208062,11206.210816,0.198342,0.206479,0.220900
min,2.725000e+05,0.000000,0.000000,0.000000,0.000000,14248.000000,-34.106240,150.553840,0.000122
25%,1.000000e+06,1.000000,3.000000,1.000000,0.308943,32500.000000,-33.921480,150.956810,0.151530
50%,1.360000e+06,2.000000,4.000000,2.000000,0.444715,39260.000000,-33.813780,151.115060,0.260955
75%,2.000000e+06,2.000000,4.000000,2.000000,0.553659,45552.000000,-33.723380,151.227530,0.482987
max,1.700000e+07,46.000000,47.000000,50.000000,0.974797,97500.000000,-33.211420,151.563810,0.957650


In [18]:
data_frame.to_csv('../Feature_Engineering/wrangled_data.csv_1', index=False)